In [1]:
from  lib.mongo_db  import db_conn
import pandas as pd
import numpy as np
from pyecharts import Bar,configure,Pie
import time

In [3]:
def date_to_time(date):
    timeArray = time.strptime(date, "%Y%m%d")
    time_semp = time.mktime(timeArray)
    return time_semp   # 2013

def time_to_today(timeStamp):
    timeArray = time.localtime(int(timeStamp))
    timeArray =time.strftime("%Y-%m-%d",timeArray)
    return timeArray   # 2013

In [134]:
def pri_tu(x,y,title):
    bar= Bar('day_anly','Theme')
    bar.add(title,x,y,is_more_utils = True,is_label_show=True, is_datazoom_show=True)
    return bar

In [4]:
def pri_pie(x,y,title):
    pie= Pie('饼图')
    pie.add('',x,y,is_label_show = True)
    return pie

In [13]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181020'),'$lte':date_to_time('20181031')}},
                           {'sid':1,'status':1,'end_time':1,'message':1,'_id':0})
data=list(data)

In [31]:
pd_data=pd.DataFrame(x for x in data)

In [32]:
pd_data['end_time']=pd_data['end_time'].apply(lambda x:time_to_today(x))
pd_data['status']=pd_data['status'].map(str)

### 10天内各失败返回码的数量分析

In [159]:
ret=pd_data.groupby(['status','message'])['sid'].count().reset_index()

In [162]:
ret[29:]

,status,message,sid
29,3098,error occurred when crawl,27
30,3099,exit by user,608
31,4000,登录失败！手机号、密码或其它参数错误。,7
32,4001,手机号错误,11
33,4002,服务密码错误,161
34,4003,验证码错误,284
35,4004,会话已超时,25
36,4004,姓名错误,16
37,4005,身份证错误,20
38,4006,禁止查询：该号码未进行实名认证,5


In [131]:
all_err=pd_data.loc[pd_data.status.values!='0',:].groupby(['status'])['sid'].count().reset_index()

In [132]:
X=all_err.status.values.tolist()
Y=all_err.sid.values.tolist()
title='err_code vs count'

In [135]:
pri_tu(X,Y,title)

In [153]:
every_day_err=pd_data.groupby(['status','end_time'])['sid'].count().reset_index()

In [154]:
def pri_err_code_count(err_code):
    one_bar= Bar(err_code,'Theme')
    filte=np.logical_and(True,every_day_err.status.values==err_code)
    filte2=np.logical_and(True,every_day_err.status.values=='0')
    all_success_count=every_day_err.loc[filte2,:]
    one_day_err=every_day_err.loc[filte,:]
    pd_ret=pd.merge(all_success_count,one_day_err,on=['end_time'],how='left')
    pd_ret=pd_ret.fillna(0)
    X=one_day_err.end_time.values.tolist()
    Y=one_day_err.sid.values.tolist()
    one_bar.add(err_code,X,Y,is_more_utils = True,is_label_show=True)
    return one_bar


In [155]:
def pri_err_code_tu(err_code):
    one_bar= Bar(err_code,'Theme')
    filte=np.logical_and(True,every_day_err.status.values==err_code)
    filte2=np.logical_and(True,every_day_err.status.values=='0')
    all_success_count=every_day_err.loc[filte2,:]
    one_day_err=every_day_err.loc[filte,:]
    pd_ret=pd.merge(all_success_count,one_day_err,on=['end_time'],how='left')
    pd_ret=pd_ret.fillna(0)
    X=pd_ret.end_time.values.tolist()
    print(err_code)
    print(pd_ret.sid_y.values)
    print(pd_ret.sid_x.values)
    Y=(pd_ret.sid_y.values/pd_ret.sid_x.values).tolist()
    one_bar.add(err_code,X,Y,is_more_utils = True,is_label_show=True)
    return one_bar

In [156]:
tu_list=[]
for err_code in every_day_err.status.unique().tolist():
    tu_list.append(pri_err_code_count(err_code))

In [157]:
len(tu_list)

46

In [158]:
tu_list.pop()

In [164]:
def today_to_day():
    timeArray = time.localtime()
    timeArray =time.strftime("%d",timeArray)
    return timeArray

In [165]:
today_to_day()

'31'